# Using Sent2Vec via Gensim

This tutorial is about using sent2vec model in Gensim. Here, we'll learn to work with the sent2vec library for training sentence-embedding models, saving & loading them and performing similarity operations.

# What is Sent2Vec?

Sent2Vec delivers numerical representations (features) for short texts or sentences, which can be used as input to any machine learning task later on. Think of it as an unsupervised version of FastText, and an extension of word2vec (CBOW) to sentences. The method uses a simple but efficient unsupervised objective to train distributed representations of sentences. The algorithm outperforms the state-of-the-art unsupervised models on most benchmark tasks, and on many tasks even beats supervised models, highlighting the robustness of the produced sentence embeddings, see the [paper](https://arxiv.org/abs/1703.02507) for more details.

The sentence embedding is defined as the average of the source word embeddings of its constituent words. This model is furthermore augmented by also learning source embeddings for not only unigrams but also n-grams present in each sentence, and averaging the n-gram embeddings along with the words

# Training model

For this tutorial, we'll be training our model using the Lee Background Corpus included in gensim. This corpus contains 314 documents selected from the Australian Broadcasting Corporation’s news mail service, which provides text e-mails of headline stories and covers a number of broad topics.

In [1]:
import gensim
import os
from gensim.models.word2vec import LineSentence
from gensim.models.sent2vec import Sent2Vec
from gensim.utils import tokenize
import re
import time
import smart_open
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Prepare training data

In [2]:
data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = data_dir + 'lee_background.cor'
lee_data = []
with smart_open.smart_open(lee_train_file) as f1, smart_open.smart_open("./input.txt",'w') as f2:
    for line in f1:
        if line not in ['\n', '\r\n']:
            line = re.split('\.|\?|\n', line.strip())
            for sentence in line:
                if len(sentence) > 1:
                    sentence = tokenize(sentence)
                    lee_data.append(list(sentence))
                    f2.write(' '.join(lee_data[-1]) + '\n')

In [3]:
# Print sample training data
for sentence in lee_data[:5]:
    print sentence,'\n'

[u'Hundreds', u'of', u'people', u'have', u'been', u'forced', u'to', u'vacate', u'their', u'homes', u'in', u'the', u'Southern', u'Highlands', u'of', u'New', u'South', u'Wales', u'as', u'strong', u'winds', u'today', u'pushed', u'a', u'huge', u'bushfire', u'towards', u'the', u'town', u'of', u'Hill', u'Top'] 

[u'A', u'new', u'blaze', u'near', u'Goulburn', u'south', u'west', u'of', u'Sydney', u'has', u'forced', u'the', u'closure', u'of', u'the', u'Hume', u'Highway'] 

[u'At', u'about', u'pm', u'AEDT', u'a', u'marked', u'deterioration', u'in', u'the', u'weather', u'as', u'a', u'storm', u'cell', u'moved', u'east', u'across', u'the', u'Blue', u'Mountains', u'forced', u'authorities', u'to', u'make', u'a', u'decision', u'to', u'evacuate', u'people', u'from', u'homes', u'in', u'outlying', u'streets', u'at', u'Hill', u'Top', u'in', u'the', u'New', u'South', u'Wales', u'southern', u'highlands'] 

[u'An', u'estimated', u'residents', u'have', u'left', u'their', u'homes', u'for', u'nearby', u'Mittago

# Using gensim implementation of sent2vec

In [4]:
# Train new sent2vec model
% time sent2vec_model = Sent2Vec(lee_data, size=100, epochs=20, seed=42, workers=4)

INFO:gensim.models.sent2vec:Creating dictionary...
INFO:gensim.models.sent2vec:Read 0.06 M words
INFO:gensim.models.sent2vec:Dictionary created, dictionary size: 1531, tokens read: 60302
INFO:gensim.models.sent2vec:training model with 4 workers on 1531 vocabulary and 100 features
INFO:gensim.models.sent2vec:PROGRESS: at 7.45% words, 65507 words/s
INFO:gensim.models.sent2vec:PROGRESS: at 16.56% words, 75960 words/s
INFO:gensim.models.sent2vec:PROGRESS: at 25.67% words, 78706 words/s
INFO:gensim.models.sent2vec:PROGRESS: at 37.26% words, 84894 words/s
INFO:gensim.models.sent2vec:PROGRESS: at 48.85% words, 88338 words/s
INFO:gensim.models.sent2vec:PROGRESS: at 61.28% words, 91325 words/s
INFO:gensim.models.sent2vec:PROGRESS: at 76.19% words, 97029 words/s
INFO:gensim.models.sent2vec:PROGRESS: at 91.09% words, 101204 words/s
INFO:gensim.models.sent2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.sent2vec:worker thread finished; awaiting finish of 2 more thr

# Training hyperparameters

Sent2Vec supports the following parameters:

- sentences : For larger corpora (like the Toronto corpus), consider an iterable that streams the sentences directly from disk/network.
- size : Dimensionality of the feature vectors. Default 100
- lr : Initial learning rate. Default 0.2
- seed : For the random number generator for reproducible reasons. Default 42
- min_count : Ignore all words with total frequency lower than this. Default 5
- max_vocab_size : Limit RAM during vocabulary building; if there are more unique words than this, then prune the infrequent ones. Every 10 million word types need about 1GB of RAM. Default is 30000000.
- t : Threshold for configuring which higher-frequency words are randomly downsampled; default is 1e-3, useful range is (0, 1e-5).
- loss_type : Default is 'ns', negative sampling will be used.
- neg : Specifies how many "noise words" should be drawn (usually between 5-20). Default is 10.
- epochs : Number of iterations (epochs) over the corpus. Default is 5.
- lr_update_rate : Change the rate of updates for the learning rate. Default is 100.
- word_ngrams : Max length of word ngram. Default is 2.
- bucket : Number of hash buckets for vocabulary. Default is 2000000.
- minn : Min length of char ngrams. Default is 3.
- maxn : Max length of char ngrams. Default is 6.
- dropout_k : Number of ngrams dropped when training a sent2vec model. Default is 2.
- batch_words : Target size (in words) for batches of examples passed to worker threads (and thus cython routines). Default is 10000. (Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
- workers : Use this many worker threads to train the model (=faster training with multicore machines). Default is 3.

In [5]:
# Print sentence vector
sent2vec_model[['This', 'is', 'an', 'awesome', 'gift']]

array([ 0.64630986,  0.49406933,  0.44399851,  0.4815839 ,  0.3750164 ,
        0.48219642,  0.57200459,  0.6533206 ,  0.54308654,  0.66870868,
        0.44194048,  0.62408952,  0.41566788,  0.58259585,  0.50704353,
        0.40808501,  0.71100687,  0.56950592,  0.75956461,  0.39850533,
        0.57919411,  0.61307228,  0.27474469,  0.63233622,  0.56929861,
        0.4429572 ,  0.59160646,  0.61236784,  0.52849234,  0.52422322,
        0.49249924,  0.54417809,  0.41359957,  0.46381369,  0.49220442,
        0.52751543,  0.5869019 ,  0.47045586,  0.262107  ,  0.62205985,
        0.73264098,  0.50952784,  0.64797648,  0.22369163,  0.28038733,
        0.74366549,  0.40995078,  0.57556881,  0.36430264,  0.34316641,
        0.40417662,  0.39607115,  0.31872905,  0.48440597,  0.78628578,
        0.46994545,  0.46389867,  0.63811896,  0.49737146,  0.61263371,
        0.32389616,  0.38991358,  0.42443156,  0.46226224,  0.40821649,
        0.45894178,  0.51032292,  0.38559933,  0.39272977,  0.46

In [6]:
# Print cosine similarity between two sentences
sent2vec_model.similarity(['This', 'is', 'an', 'awesome', 'gift'], ['This', 'present', 'is', 'great'])

0.96225385104138639

# Saving and loading models

Models can be saved and loaded via the load and save methods.

In [7]:
# Save trained sent2vec model
sent2vec_model.save('s2v1')

INFO:gensim.utils:saving Sent2Vec object under s2v1, separately None
INFO:gensim.utils:storing np array 'wi' to s2v1.wi.npy
INFO:gensim.utils:saved s2v1


In [9]:
# Load pretrained sent2vec model
loaded_model = Sent2Vec.load('s2v1')

INFO:gensim.utils:loading Sent2Vec object from s2v1
INFO:gensim.utils:loading wi from s2v1.wi.npy with mmap=None
INFO:gensim.utils:loaded s2v1
